In [95]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [96]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [97]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

In [98]:
import helpers_self

In [99]:
cpu_device = 'cpu'
cpu_device

gpu_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
gpu_device

'cpu'

device(type='cuda', index=0)

In [100]:
eval_size = 0.2
batch_size = 32
num_workers = 0

In [101]:
transform = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                           ])

In [102]:
train_dataset = datasets.FashionMNIST(root = 'data',
                                       train = True,
                                       download = True,
                                       transform = transform)

test_dataset = datasets.FashionMNIST(root = 'data',
                                       train = True,
                                       download = True,
                                       transform = transform)

In [103]:
train_dataset_len = len(train_dataset)
train_dataset_len

60000

In [ ]:
indices = list(range(train_dataset_len))
indices[:5]

[0, 1, 2, 3, 4]

In [ ]:
np.random.shuffle(indices)
indices[:5]

[16497, 54185, 20613, 57826, 20526]

In [ ]:
split = int(np.floor(train_dataset_len * eval_size))
train_idx, eval_idx = indices[split:], indices[:split]

In [ ]:
len(train_idx)

48000

In [ ]:
len(eval_idx)

12000

In [ ]:
train_sampler = SubsetRandomSampler(train_idx)
eval_sampler = SubsetRandomSampler(eval_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           sampler = train_sampler,
                                           num_workers = num_workers
                                          )

eval_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          sampler = eval_sampler,
                                          num_workers = num_workers
                                         )

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = num_workers,
                                          
                                          
                                         )

In [ ]:
# specify the image classes
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

In [ ]:
images.to(cpu_device)
labels.to(cpu_device)

In [ ]:
type(images)
images.shape

In [ ]:
images.shape

In [ ]:
type(labels)
labels.shape
labels

In [ ]:
#labels = [item.get_text() for item in ax.get_xticklabels()]
#labels[1] = 'Testing'

#ax.set_xticklabels(labels)

In [ ]:
InteractiveShell.ast_node_interactivity = 'none'

In [ ]:
plt.figure(figsize = (55, 30))
for idx in range(batch_size):
    ax = plt.subplot(4, 8, idx + 1, xticks = [], yticks = [])
    helpers_self.imshow(ax, images[idx, :], classes[labels[idx].item()])

In [ ]:
class Cnn_fashion_mnist(nn.Module):
    
    def __init__(self):
        super(Cnn_fashion_mnist, self).__init__()

        self.hidden_layer_i = nn.Linear(784, 256)

        self.hidden_layer_ii = nn.Linear(256, 128)

        self.output = nn.Linear(128, 10)
        
        
    def forward(self, x):

        x = F.relu(self.hidden_layer_i(x))

        x = F.relu(self.hidden_layer_ii(x))

        x = self.output(x)
        
        return x

In [ ]:
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
model = Cnn_fashion_mnist()
model.to(gpu_device)
model

In [ ]:
optimizer = optim.Adagrad(model.parameters(), lr = 0.01)
optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion

In [ ]:
InteractiveShell.ast_node_interactivity = 'none'

In [ ]:
epochs = 1

In [ ]:
for epoch in range(epochs):
    
    training_loss = 0.0
    eval_loss = 0.0
    
    model.train()
    for epoch_id, (images, labels) in enumerate(train_loader):
        images, labels = images.to(gpu_device), labels.to(gpu_device)
        images = images.view(batch_size, -1)

        optimizer.zero_grad()

        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        training_loss += loss.item() * batch_size
    else:
        with torch.no_grad():
            model.eval()

            for eval_batch_id, (images, labels) in enumerate(eval_loader):

                images, labels = images.to(gpu_device), labels.to(gpu_device)
                images = images.view(batch_size, -1)

                output = model.forward(images)
                loss = criterion(output, labels)

                eval_loss += loss.item() * batch_size
    
    avg_training_loss = training_loss/len(train_loader)
    avg_eval_loss = eval_loss/len(eval_loader)
    
    print(
    "Epoch {} : Training Loss {:.5f} | Eval Loss {:.5f}".format(
        epoch + 1,
        avg_training_loss,
        avg_eval_loss
        )
    )

In [ ]:
with torch.no_grad():
    model.eval()

    test_iter = iter(test_loader)
    test_images, test_labels = next(test_iter)
    test_images, test_labels = test_images.to(gpu_device), test_labels.to(gpu_device)
    test_images = test_images.view(batch_size, -1)
    
    output = model.forward(test_images)
    _, predicted_class = torch.max(input = output, dim = 1)
    
    test_images, test_labels = test_images.to(cpu_device), test_labels.to(cpu_device).numpy()
    predicted_class = predicted_class.to(cpu_device).numpy()
    
    equals = predicted_class == test_labels
    accuracy = np.mean(equals)
    
    print('Accuracy on Test Dataset {}'.format(accuracy))

In [ ]:
plt.figure(figsize = (40, 40))
for index in range(batch_size):
    
    ax = plt.subplot(4, 8, index + 1, xticks = [], yticks = [])
    img = test_images[index].view(1, -1).numpy()

    ax.imshow(img, cmap = 'gray')